# Pandas groupby 集計 高速化

[Pandas groupby 集計を10倍高速化する3つの基本テクニック](https://www.salesanalytics.co.jp/datascience/datascience268/)


In [1]:
import pandas as pd

In [2]:
import pandas as pd
# まずは6行だけの小さなデータで試してみます
df_sample = pd.DataFrame({
    'region':  ['関東', '関東', '関西', '関西', '関西', '中部'],
    'channel': ['店舗', 'オンライン', '店舗', 'オンライン', '店舗', 'オンライン'],
    'sales':   [100, 200, 150, 80, 120, 90]
})
# データの中身を確認
print(df_sample)
 

  region channel  sales
0     関東      店舗    100
1     関東   オンライン    200
2     関西      店舗    150
3     関西   オンライン     80
4     関西      店舗    120
5     中部   オンライン     90


In [3]:
# 地域ごとの売上合計を計算
result = df_sample.groupby('region')['sales'].sum()
print(result)

region
中部     90
関東    300
関西    350
Name: sales, dtype: int64


## 複数の条件でグループ化する

In [4]:
# 地域×チャネル別の平均売上を計算
# as_index=Falseを付けると、結果が見やすい表形式になります
result = df_sample.groupby(['region', 'channel'], as_index=False)['sales'].mean()
print(result)

  region channel  sales
0     中部   オンライン   90.0
1     関東   オンライン  200.0
2     関東      店舗  100.0
3     関西   オンライン   80.0
4     関西      店舗  135.0


## 高速化

### 検証用データの準備

In [5]:
import numpy as np
import pandas as pd
import time
# 乱数のシードを固定
np.random.seed(42)
# 100万行のデータを生成
N = 1_000_000  # アンダースコアで区切ると読みやすい
regions = ["関東", "関西", "中部", "東北", "九州"]
channels = ["オンライン", "店舗", "パートナー", "電話"]
df = pd.DataFrame({
    "region": np.random.choice(regions, size=N),    # 地域をランダムに選択
    "channel": np.random.choice(channels, size=N),   # チャネルをランダムに選択
    "sales": np.random.normal(500000, 10000, size=N).round(3)  # 平均500000、標準偏差10000の正規分布
})
print(f"データのサイズ: {len(df):,}行")  # カンマ区切りで表示
print(df.head())  # 最初の5行を確認

データのサイズ: 1,000,000行
  region channel       sales
0     東北   オンライン  497092.948
1     九州   パートナー  482762.557
2     中部      店舗  497356.177
3     九州      店舗  494995.646
4     九州      電話  490321.006


### 最適化前の処理速度を測定

In [6]:
# データ型を確認（regionとchannelがobject型になっているはず）
print("現在のデータ型:")
print(df.dtypes)
print()
# メモリ使用量を確認
memory_mb = df.memory_usage(deep=True).sum() / 1024**2
print(f"メモリ使用量: {memory_mb:.2f} MB")
print()
# 速度を表示
start_time = time.perf_counter()  # 正確な時間測定用のタイマーを開始
result_before = df.groupby(["region", "channel"])['sales'].agg(['sum', 'mean', 'count'])
time_before = time.perf_counter() - start_time
print(f"処理時間（最適化前）: {time_before:.3f} 秒")

現在のデータ型:
region      object
channel     object
sales      float64
dtype: object

メモリ使用量: 174.52 MB

処理時間（最適化前）: 0.120 秒


### データ型を最適化して再測定

In [7]:
# データをコピー（元のデータは残しておく）
df_optimized = df.copy()
# カテゴリ型に変換
df_optimized["region"] = df_optimized["region"].astype("category")
df_optimized["channel"] = df_optimized["channel"].astype("category")
print("最適化後のデータ型:")
print(df_optimized.dtypes)
print()
# メモリ使用量を再確認
memory_mb_optimized = df_optimized.memory_usage(deep=True).sum() / 1024**2
print(f"メモリ使用量: {memory_mb_optimized:.2f} MB")
print(f"メモリ削減率: {(1 - memory_mb_optimized/memory_mb)*100:.1f}%")
print()
# 最適化された処理の実行
start_time = time.perf_counter()
result_after = df_optimized.groupby(
    ["region", "channel"], 
    observed=True,  # 実際に存在するグループだけを処理
    sort=False      # 並べ替えをスキップ
)['sales'].agg(['sum', 'mean', 'count'])
# 速度を表示
time_after = time.perf_counter() - start_time
print(f"処理時間（最適化後）: {time_after:.3f} 秒")
print(f"高速化率: {time_before / time_after:.1f}倍")

最適化後のデータ型:
region     category
channel    category
sales       float64
dtype: object

メモリ使用量: 9.54 MB
メモリ削減率: 94.5%

処理時間（最適化後）: 0.033 秒
高速化率: 3.6倍
